# load data

In [1]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [2]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Normal\50.jpg,1
1,../dataset/wrist_xray/Fracture\2 Male (A View)...,0
2,../dataset/wrist_xray/Normal\41.jpg,1
3,../dataset/wrist_xray/Fracture\46 Male (A View...,0
4,../dataset/wrist_xray/Normal\68.jpg,1


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [3]:
import sys
sys.path.append("../src") 

In [4]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel4
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

d:\tutorials\automl\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print('MyNet parameters :' ,count_parameters(mymodel4()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 0.391408
SqueezeNet parameters : 0.787008
ShuffleNet parameters : 0.47256
MnastNet parameters : 1.101192
MobileNet parameters : 2.387264
DenseNet parameters : 7.078784


In [7]:
epoch=32
batchsize=8
lr=0.001

dim='3D' or '2D' for 3D images and 2D iamges
skip_tuning=False, no grid search applied for hyper-parameter tuning, 
aug=1-> pytorch augmentation
aug=0-> no augmentation
aug=2-> albumentation augmentation
aug=3-> rand augmentation

# MyNet

In [8]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel4(),epochs=epoch,batchsize=batchsize,data=data,\
                            lr=0.001,skip_tuning=False,aug=1,dim='2D',pca=False)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.872   0.782     0.820
Class 1       0.750   0.836     0.786
macro         0.810   0.812     0.804
weighted      0.820   0.810     0.806
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.884   0.836     0.858
Class 1       0.800   0.852     0.822
macro         0.842   0.844     0.840
weighted      0.850   0.846     0.844
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.874   0.828     0.848
Class 1       0.790   0.840     0.810
macro         0.834   0.834     0.830
weighted      0.842   0.836     0.836
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.896   0.864     0.880
Class 1       0.822   0.866     0.844
macro         0.858   0.862     0.860
weighted      0.864   0.864     0.864
--------------------------Accuracy Tab

,DT,SVM,LR,KNN,ML
0,0.87,0.92,0.90,0.92,0.95
1,0.72,0.77,0.77,0.79,0.79
2,0.82,0.85,0.85,0.90,0.90
3,0.82,0.87,0.84,0.87,0.87
4,0.82,0.82,0.82,0.84,0.84
mean,0.81,0.846,0.836,0.864,0.87


In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel4(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

# SqueezeNet

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

# ShuffleNet

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

# Mnasnet

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

# MobileNet

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

# DenseNet121

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

In [ ]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]